In [29]:
#Importing Package
#importing
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
import matplotlib.pyplot as plt
import os
print("all libraries are loaded!")

all libraries are loaded!


In [30]:
# INPUT Import the data (After stop identifying program)
folder = 'pitt-DriverID_cal' 
filename = '4326a593-9272-4411-b3b7-05555cca778420220417_162338'
driver = filename[0:36]
df = pd.read_csv(folder + '/' + filename+'.csv')

#Format  DateTime
df["DateTime"] = pd.to_datetime(df['DateTime'])

print(len(df))
df.head()

992


,driver_id,shift_id,lat,lon,accuracy,speed,heading,activity_type,activity_confidence,DateTime,time_diffs,speed_mean,Stop_check,Stop_real,O_or_D
0,4326a593-9272-4411-b3b7-05555cca7784,347303.0,40.491580,-80.264281,4.0,0.0,208.25,still,33,2019-08-21 17:49:11.998,0.000,0.000000,1.0,1.0,Origin
1,4326a593-9272-4411-b3b7-05555cca7784,NaN,40.491580,-80.264281,4.0,0.0,208.25,unknown,100,2019-08-21 17:49:11.998,0.000,NaN,NaN,1.0,Origin
2,4326a593-9272-4411-b3b7-05555cca7784,NaN,40.491580,-80.264281,4.0,0.0,208.25,unknown,100,2019-08-21 17:49:12.372,0.374,0.000000,1.0,1.0,Origin
3,4326a593-9272-4411-b3b7-05555cca7784,347303.0,40.491583,-80.264277,4.0,0.0,208.41,still,33,2019-08-21 17:49:46.997,34.625,0.013098,1.0,1.0,Origin
4,4326a593-9272-4411-b3b7-05555cca7784,347304.0,40.491583,-80.264277,4.0,0.0,208.41,still,33,2019-08-21 17:49:46.997,0.000,NaN,NaN,1.0,Origin


In [31]:
df['Stop_real'].iloc[1]

1.0

In [32]:
df['DateTime'].iloc[1]

Timestamp('2019-08-21 17:49:11.998000')

In [33]:
np.sin(np.pi/2.+2*np.pi)

1.0

In [34]:
# for each row in df
for i in range(len(df)):
    if i == 0:
        n = 0
        FR_or_TO = np.sin(np.pi/2.+n*np.pi) # sin(pi/2) = 1
        # make list for FROM Lat , Lon, DateTime
        FR_LAT = []
        FR_LON = []
        FR_TIM = []
        # make list for TO Lat , Lon, DateTime
        TO_LAT = []
        To_LON = []
        TO_TIM = []
    if i > 0:
        # if stop_real row i isnt same as row i-1 AND is 1
        if df['Stop_real'].iloc[i] != df['Stop_real'].iloc[i-1] and df['Stop_real'].iloc[i] == 1 :
            if FR_or_TO == 1: #From
                FR_LAT.append(df['lat'].iloc[i])
                FR_LON.append(df['lon'].iloc[i])
                FR_TIM.append(df['DateTime'].iloc[i])
            if FR_or_TO == -1: #To
                TO_LAT.append(df['lat'].iloc[i])
                To_LON.append(df['lon'].iloc[i])
                TO_TIM.append(df['DateTime'].iloc[i])

        # if stop_real row i isnt same as row i-1 AND is 0
        if df['Stop_real'].iloc[i] != df['Stop_real'].iloc[i-1] and df['Stop_real'].iloc[i] == 0 :
            if FR_or_TO == 1: #From
                n = n +1
                FR_or_TO = np.sin(np.pi/2.+n*np.pi)
            if FR_or_TO == -1: #To
                FR_LAT.append(df['lat'].iloc[i-1])
                FR_LON.append(df['lon'].iloc[i-1])
                FR_TIM.append(df['DateTime'].iloc[i-1])
while len(FR_TIM) > len(TO_TIM):
    del FR_LAT[-1]
    del FR_LON[-1]
    del FR_TIM[-1]
print(len(FR_LAT))
print(len(FR_LON))
print(len(FR_TIM))
print(len(TO_LAT))
print(len(To_LON))
print(len(TO_TIM))

4
4
4
4
4
4


In [35]:
TO_TIM

[Timestamp('2019-08-21 17:51:52.887000'),
 Timestamp('2019-08-21 19:02:09.994000'),
 Timestamp('2019-08-21 19:13:56.997000'),
 Timestamp('2019-08-21 19:18:33')]

In [36]:
SequenceTable = pd.DataFrame()
SequenceTable['FR_LAT'] = FR_LAT
SequenceTable['FR_LON'] = FR_LON
SequenceTable['FR_TIM'] = FR_TIM
SequenceTable['TO_LAT'] = TO_LAT
SequenceTable['To_LON'] = To_LON
SequenceTable['TO_TIM'] = TO_TIM
SequenceTable['DRIVER_ID'] = driver
delta_t = []
for i in range(len(TO_TIM)):
    delta_t.append((TO_TIM[i]-FR_TIM[i]).total_seconds())
SequenceTable['DURA_S'] = delta_t

print(len(SequenceTable))
SequenceTable.head()

4


,FR_LAT,FR_LON,FR_TIM,TO_LAT,To_LON,TO_TIM,DRIVER_ID,DURA_S
0,40.491584,-80.264277,2019-08-21 17:49:54.997,40.491589,-80.264294,2019-08-21 17:51:52.887,4326a593-9272-4411-b3b7-05555cca7784,117.890
1,40.492371,-80.265818,2019-08-21 18:31:47.997,40.512369,-80.097151,2019-08-21 19:02:09.994,4326a593-9272-4411-b3b7-05555cca7784,1821.997
2,40.512469,-80.097204,2019-08-21 19:02:34.994,40.521488,-80.033866,2019-08-21 19:13:56.997,4326a593-9272-4411-b3b7-05555cca7784,682.003
3,40.521405,-80.033640,2019-08-21 19:14:18.996,40.521094,-80.025103,2019-08-21 19:18:33.000,4326a593-9272-4411-b3b7-05555cca7784,254.004


In [37]:
# To Create Directory If Not Exist
# https://appdividend.com/2021/07/03/how-to-create-directory-if-not-exist-in-python/
path = 'Trip/TripSequencing'
if not os.path.exists(path):
    os.makedirs(path, exist_ok=False)
# datetime object containing current date and time
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")
#export
SequenceTable.to_csv(path+ "/" + 'T_'+filename + dt_string + ".csv",index = 0)